<a href="https://colab.research.google.com/github/okuokuch/pokemon_home/blob/main/pokemon_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon HOMEのシーズン情報取得

In [29]:
import requests
import json
import pandas as pd

#データ取得のためのダミー設定値。スマホからアクセスしているような設定。
headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'countrycode': '304',
    'authorization': 'Bearer',
    'langcode': '1',
    'user-agent': 'Mozilla/5.0 (Linux; Android 8.0; Pixel 2 Build/OPD3.170816.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Mobile Safari/537.36',
    'content-type': 'application/json'
}

data = '{"soft":"Sw"}'
#リクエストを出してシーズン情報を取得
response_base = requests.post('https://api.battle.pokemon-home.com/cbd/competition/rankmatch/list', headers=headers, data=data)
info_all = json.loads(response_base.text)

In [22]:
#@title 取り出したい情報 { display-mode: "both" }
#seasonには数字を入力して下し
#ruleにはシングルなら1、ダブルなら0を入力してください。
season = "30" #@param {type:"string"}
rule = 0 #@param {type:"integer"}


In [41]:
#指定したシーズン情報を取り出すためのリクエストパラメータ取得
info_season = info_all['list'][season]

for content in info_season:
  if info_season[content]['rule'] == rule:
    params = {'id':content, 'rst':str(info_season[content]['rst']), 'ts1':str(info_season[content]['ts1']), 'ts2':str(info_season[content]['ts2'])}

In [45]:
for x in range(1,2):
    adress = "https://resource.pokemon-home.com/battledata/ranking/"+params["id"]+"/"+params["rst"]+"/"+params["ts2"]+"/pdetail-"+str(x)
    headers = {
        'user-agent': 'Mozilla/5.0 (Linux; Android 8.0; Pixel 2 Build/OPD3.170816.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Mobile Safari/537.36',
        'content-type': 'application/json',
    }

    response = requests.get(adress, headers=headers)
    json_read= json.loads(response.text)